# Load packages

In [1]:
#Load libs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from tysserand import tysserand as ty
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 
from PIL import Image, ImageOps
import fcsparser

from os import listdir
from os.path import isfile, join
#set up working dir

import sys
sys.path.extend([
    '../tysserand/tysserand',
    '../mosna',
])

import seaborn as sns
from time import time
import copy
from skimage import color
import matplotlib as mpl
import napari

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from scipy.stats import loguniform

import umap
# if not installed run: conda install -c conda-forge umap-learn
import hdbscan
from sklearn.cluster import OPTICS, cluster_optics_dbscan

from skimage import io

from scipy.stats import ttest_ind    # Welch's t-test
from scipy.stats import mannwhitneyu # Mann-Whitney rank test
from scipy.stats import ks_2samp     # Kolmogorov-Smirnov statistic

sys.path.append("/home/mouneem/mosna/")
from collections import Counter


from tysserand import tysserand as ty
from mosna import mosna
import glob
import re

## Load data

In [16]:
# GENERATE CSV FILE OF EACH LAYERS
pathC1 = '/mnt/SERVER-CRCT-STORAGE/CRCT_Imagin/CORDELIER Pierre/HaloData/21-003.IMMCORE.C2v1/Halo archive 2021-12-13 16-38 - v3.3.2541/ObjectData/'
#pathC1 = '/mnt/SERVER-CRCT-STORAGE/CRCT_Imagin/CORDELIER Pierre/HaloData/21-003.IMMCORE.C1v1/Halo archive 2021-12-13 13-32 - v3.3.2541/ObjectData/'
outpath = '/home/mouneem/PNETS-NETS/Data/'
csvs = [f for f in listdir(pathC1) if isfile(join(pathC1, f))]

for csvFile in csvs:
    csvData = pd.read_csv(pathC1 + csvFile)
    csvData[['x']] = ( csvData[['XMax']] + csvData[['XMax']] ) / 2
    csvData[['y']] = ( csvData[['YMax']] + csvData[['YMax']] ) / 2

    col_is_marker = []
    for col in list(csvData.columns):
        if col.count('+') + col.count('-') > 1:
            col_is_marker.append(True)
        else:
            col_is_marker.append(False)
    marker = csvData.loc[:, col_is_marker]
    
    ################# COMBO 1
    if 'C2v1' in pathC1: 
        C1_markers = marker['PanCK+CD3-CD8-CD20-'].astype(bool) 
        C2_markers = marker['PanCK-CD3+CD8+'].astype(bool) 
        C3_markers = marker['PanCK-CD3+CD8-'].astype(bool) 
        C4_markers = marker['PanCK-CD3-CD20+'].astype(bool) 
        C5_markers = marker['PanCK-CD3+CD20+'].astype(bool) 
        
        classes = ['Other']*csvData.shape[0]
        for i in range(len(classes)):
            if C1_markers.values[i]:
                classes[i] = 'C1'
            elif C2_markers.values[i]:
                classes[i] = 'C2'
            elif C3_markers.values[i]:
                classes[i] = 'C3'
            elif C4_markers.values[i]:
                classes[i] = 'C4'
            elif C5_markers.values[i]:
                classes[i] = 'C5'
                
    ################# COMBO 2 : 'CD206+CD163+CD68-CD16-', 'CD206+CD163-CD68+CD16-', 'CD206+CD163-CD68-CD16+', 'CD206-CD163+CD68+CD16-', 'CD206-CD163+CD68-CD16+', 'CD206-CD163-CD68+CD16+', 'CD206+CD163+CD68+CD16-', 'CD206+CD163+CD68-CD16+', 'CD206-CD163+CD68+CD16+', 'CD206+CD163-CD68+CD16+', 'CD206+CD163+CD68+CD16+'
    elif 'C1v1' in pathC1: 
        M1_markers = marker['CD206-CD163-CD68+CD16+'].astype(bool)
        M2_markers = (marker['CD206+CD163+CD68-CD16+'] + marker['CD206+CD163+CD68-CD16-']).astype(bool) 
        TAM_markers = (marker['CD206+CD163+CD68+CD16-'] + marker['CD206+CD163+CD68+CD16+']).astype(bool) 
        classes = ['Other']*csvData.shape[0]
        for i in range(len(classes)):
            if M1_markers.values[i]:
                classes[i] = 'M1'
            elif M2_markers.values[i]:
                classes[i] = 'M2'
            elif TAM_markers.values[i]:
                classes[i] = 'TAM'
        

    csvData["class"] = classes
    nodes = csvData[['x','y',"class",'Image Location']]
    if (len(np.unique(csvData.iloc[:, 1].value_counts() ))==1):
        outname = csvData.iloc[1, 0].split('\\')[-1] #FILE NAME
        outname = outname + '_' + csvData.iloc[1, 1] + '.csv'  # LAYER : csvData.iloc[1, 1]
        nodes.to_csv(outpath + "csv_levels/" + outname,  mode='a', header = False)


# Generate network for one layer